In [0]:
npi_df = spark.read.parquet("/mnt/bronze/npi_extract")
npi_df.createOrReplaceTempView("npi")

In [0]:
%sql
select * from npi

In [0]:
%sql
create or replace temp view quality_checks
as 
select *, 
case when npi_id is null then true
else false
end as is_quarantined
from npi 

In [0]:
%sql
select * from quality_checks

In [0]:
%sql
create table if not exists silver.npi_ext(
    first_name string,
    last_name string,
    last_updated date,
    npi_id string,
    organisation_name string,
    position string,
    refreshed_at date,
    is_quarantined boolean,
    inserted_date timestamp,
    modified_date timestamp,
    is_current boolean
)

In [0]:
%sql
merge into silver.npi_ext as target
using quality_checks as source
on target.npi_id = source.npi_id
and is_current = true
when matched and
(
target.first_name <> source.first_name
or target.last_name <> source.last_name
or target.last_updated <> source.last_updated
or target.organisation_name <> source.organisation_name
or target.position <> source.position
or target.refreshed_at <> source.refreshed_at
or target.is_quarantined <> source.is_quarantined
)
then update set
target.modified_date = current_timestamp(),
target.is_current = false
when not matched then insert(
    first_name,
    last_name,
    last_updated,
    npi_id,
    organisation_name,
    position,
    refreshed_at,
    is_quarantined,
    inserted_date,
    modified_date,
    is_current
) values(
    source.first_name,
    source.last_name,
    source.last_updated,
    source.npi_id,
    source.organisation_name,
    source.position,
    source.refreshed_at,
    source.is_quarantined, 
    current_timestamp(),
    current_timestamp(),
    true
)

In [0]:
%sql
select * from silver.npi_ext